
# Getting Started

ML Ops is gaining a lot of popularity. This example showcases a key piece you can use to construct your automation pipeline. As we can see in the following architecture diagram, you will be deploying an AWS Step Funciton Workflow containing AWS Lambda functions that call Amazon S3, Amazon Personalize, and Amazon SNS APIs.


This package contains the source code of a Step Functions pipeline that is able to perform 
multiple actions within **Amazon Personalize**, including the following:

- Dataset Group creation
- Datasets creation and import
- Solution creation
- Solution version creation
- Campaign creation

Once the steps are completed, the step functions notifies the users of its completion through the
use of an SNS topic.

The below diagram describes the architecture of the solution:

![Architecture Diagram](../../static/imgs/ml_ops_architecture.png)

The below diagram showcases the StepFunction workflow definition:

![stepfunction definition](../../static/imgs/step_functions.png)



## Prerequisites

### Installing AWS SAM

The AWS Serverless Application Model (SAM) is an open-source framework for building serverless applications. It provides shorthand syntax to express functions, APIs, databases, and event source mappings. With just a few lines per resource, you can define the application you want and model it using YAML. During deployment, SAM transforms and expands the SAM syntax into AWS CloudFormation syntax, enabling you to build serverless applications faster.

**Install** the [AWS SAM CLI](https://docs.aws.amazon.com/serverless-application-model/latest/developerguide/serverless-sam-cli-install.html). 
This will install the necessary tools to build, deploy, and locally test your project. In this particular example we will be using AWS SAM to build and deploy only. For additional information please visit our [documentation](https://docs.aws.amazon.com/serverless-application-model/latest/developerguide/what-is-sam.html).


**Note:** We have pre-installed SAM CLI in this notebooks through a cloudformation life cycle policy config

Let's check what version of SAM we have installed



In [ ]:
!sam --version

## Directory Structure

Let's take a look at directory structure 

We have a couple artifacts that we will be using to build our MLOps pipeline.

In [ ]:
!ls /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops

**`ml_ops/domain`**
* This directory contains the configuration file and sample data based on the domain. In this example we are going to be using the Retail domain

In [ ]:
!ls /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops/domain

**`ml_ops/lambdas`**
* This directory contains all the code that will be going into the lambda functions, these lambda functions will become a step inside the stepfunctions state machine we will deploy


In [ ]:
!ls /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops/lambdas

**`ml_ops/template.yaml`**
* This is our SAM template that will deploy the automation into our account, here we are printing just the head

In [ ]:
!head /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops/template.yaml

## Deploying

In order to deploy the project you will need to run the following commands:

In [ ]:
!cd /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops/; sam build

In [ ]:
!cd /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops/; sam deploy --template-file template.yaml --stack-name notebook-automation --capabilities CAPABILITY_IAM --s3-bucket $(aws cloudformation describe-stack-resources --stack-name AmazonPersonalizeImmersionDay --logical-resource-id SAMArtifactsBucket --query "StackResources[0].PhysicalResourceId" --output text)

## Uploading data

Let's get the bucket that our cloudformation deployed. We will be uploading our data to this bucket, plus the configuration file to trigger the automation

In [ ]:
bucket = !aws cloudformation describe-stacks --stack-name notebook-automation --query "Stacks[0].Outputs[?OutputKey=='InputBucketName'].OutputValue" --output text
bucket_name = bucket[0]
print(bucket_name)

Now that we have the bucket name, lets copy over our Media data so we can explore and upload to S3

In [ ]:
!cp -R /home/ec2-user/SageMaker/amazon-personalize-immersion-day/automation/ml_ops/domain/CPG ./example

In [ ]:
# Import Dependencies

import boto3
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import requests
import csv
import sys
import botocore
import uuid
from collections import defaultdict
import random
import numpy as np

from packaging import version
from botocore.exceptions import ClientError
from pathlib import Path

%matplotlib inline

# Setup Clients

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client('personalize-events')

# We will upload our training data in these files:
raw_items_filename = "example/data/Items/items.csv"                # Do Not Change
raw_users_filename = "example/data/Users/users.csv"                # Do Not Change
raw_interactions_filename = "example/data/Interactions/interactions.csv"  # Do Not Change
items_filename = "items.csv"                # Do Not Change
users_filename = "users.csv"                # Do Not Change
interactions_filename = "interactions.csv"  # Do Not Change


In [ ]:
interactions_df = pd.read_csv(raw_interactions_filename)
interactions_df.head()

There are 2 ways of uploading your datasets to S3:
1. Using the boto3 SDK
1. Using the CLI

In this example we are going to use the CLI command

In [ ]:
!aws s3 sync ./example/data s3://$bucket_name

## Starting the State Machine Execution

In order to execute the MLOps pipeline we need to provide a parameters file that will tell our state machine which names and configurations we want in our Amazon Personalize deployment.

We have prepared a parameters.json file, let's explore it

In [ ]:
with open('example/params.json') as f:
  data = json.load(f)

print(json.dumps(data, indent=4, sort_keys=True, default=str))

This parameters file is set up to run at the beginning of this workshop. So let's modify a couple fields to make sure we are not overwritting our previous deployment 

In [ ]:
# Dataset Groups
data['datasetGroup']['name'] = 'notebook-automation'
 
# Datasets 
data['datasets']['Interactions']['name'] = 'na-interactions-ds'
data['datasets']['Users']['name'] = 'na-users-ds'
data['datasets']['Items']['name'] = 'na-items-ds'

# Solutions

data['solutions']['personalizedRanking']['name'] = 'na-personalizedRankingCampaign'
data['solutions']['sims']['name'] = 'na-simsCampaign'
data['solutions']['userPersonalization']['name'] = 'na-userPersonalizationCampaign'

# Campaigns

data['campaigns']['personalizedRankingCampaign']['name'] = 'na-personalizedRankingCampaign'
data['campaigns']['simsCampaign']['name'] = 'na-simsCampaign'
data['campaigns']['userPersonalizationCampaign']['name'] = 'na-userPersonalizationCampaign'

# Event Tracker

data['eventTracker']['name'] = 'na-eventTracker'

print(json.dumps(data, indent=4, sort_keys=True))

## Updating and uploading your parameters file to S3

First let's write the file locally

In [ ]:
with open('example/params.json', 'w') as outfile:
    json.dump(data, outfile)

Now we can upload this file to S3, we are going to be using the CLI to do so

In [ ]:
!aws s3 cp ./example/params.json s3://$bucket_name

## Validating the deployment

So far we have deployed the automation required lets take a look at the stepfunctions execution

In [ ]:
client = boto3.client('stepfunctions')
stateMachineArn = !aws cloudformation describe-stacks --stack-name notebook-automation --query "Stacks[0].Outputs[?OutputKey=='DeployStateMachineArn'].OutputValue" --output text
stateMachineArn= stateMachineArn[0]

In [ ]:
describe_response = client.describe_state_machine(
    stateMachineArn=stateMachineArn
)
print(json.dumps(describe_response, indent=4, sort_keys=True, default=str))

In [ ]:
executions_response = client.list_executions(
    stateMachineArn=stateMachineArn,
    statusFilter='SUCCEEDED',
    maxResults=2
)
print(json.dumps(executions_response, indent=4, sort_keys=True, default=str))

### Let's look at the succeeded execution

Once your step functions are done executing, you can list the executions and describe them

In [ ]:
executions_response = client.list_executions(
    stateMachineArn=stateMachineArn,
    statusFilter='SUCCEEDED',
    maxResults=2
)
print(json.dumps(executions_response, indent=4, sort_keys=True, default=str))

In [ ]:
describe_executions_response = client.describe_execution(
    executionArn=executions_response['executions'][0]['executionArn']
)
print(json.dumps(describe_executions_response, indent=4, sort_keys=True, default=str))

### Let's look at the input that was delivered to the State Machine

As we can see below, this is the input from our Parameters file we uploaded to S3. This input json was then passed to lambda functions in the state machine to utilize across Amazon Personalize APIs

In [ ]:
print(json.dumps(json.loads(describe_executions_response['input']), indent=4, sort_keys=True, default=str))

### Let's look at the time stamps

As we can see below, this is the input from our Parameters file we uploaded to S3. This input json was then passed to lambda functions in the state machine to utilize across Amazon Personalize APIs

In [ ]:
print("Start Date:")
print(json.dumps(describe_executions_response['startDate'], indent=4, sort_keys=True, default=str))
print("Stop Date:")
print(json.dumps(describe_executions_response['stopDate'], indent=4, sort_keys=True, default=str))
print("Elapsed Time: ")
elapsed_time = describe_executions_response['stopDate'] - describe_executions_response['startDate']
print(elapsed_time)

As we see above, the automation around an hour with fourty minutes.

If you are interested in deploying this example in your environment, visit our [Github Samples Page](https://github.com/aws-samples/amazon-personalize-samples/tree/master/next_steps/operations/ml_ops) to download the latest codebase.